# Описание проекта

Добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Определение лучшей локации для разведки и добычи нефти

### Задача: построить модель для определения региона, где добыча нефти принесёт наибольшую прибыль.
### Условия задачи:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### Исходные данные: предоставлены пробы нефти в трёх регионах, характеристики для каждой скважины в регионе.

### Описание данных
* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

# Оглавление
### [1. Загрузка и подготовка данных](#1-bullet)
### [2. Обучение и проверка модели](#2-bullet)
### [3. Подготовка к расчёту прибыли](#3-bullet)
### [4. Расчёт прибыли и рисков](#4-bullet)
### [Вывод](#5-bullet)

In [1]:
# Импортирт необходимых библиотек 
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from numpy.random import RandomState
from scipy import stats as st

In [2]:
def info(data):
    """
    Функция пробегает по признакам датафрейма и если тип у признака object, то смотрит уникальные значения, в противном случае 
    оценивается распределение
    """
    for column in data.columns:   
        if data[column].dtype == 'object': 
            print(column)
            print(data[column].unique())
            print()
            print(data[column].describe())
            print()            
        else:            
            print(column)
            import plotly.express as px
            fig = px.histogram(data, x = column, marginal = 'box', title = 'Распределение '+ column)
            fig.show()

# 1. Загрузка и подготовка данных<a id='1-bullet'></a>

In [3]:
# Оценим распределение данных в признаках по скважинам в регионе №0
df_0 = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/7. Модель по выбору региона где добывать нефть выгоднее/geo_data_0.csv')
display(df_0.head()) 
#df_0.info()
#info(df_0)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
# Очистим исходные данные от выбрасов по признаку f2
df_0 = df_0[(df_0['f2'] >= -6.352956) & (df_0['f2'] <= 11.3538)]
df_0.shape

(99492, 5)

In [5]:
# Оценим распределение данных в признаках по скважинам в регионе №1
df_1 = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/7. Модель по выбору региона где добывать нефть выгоднее/geo_data_1.csv')
display(df_1.head())
#df_1.info()
#info(df_1)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
# Очистим исходные данные от выбрасов по признакам f0, f1
df_1 = (df_1[
       (df_1['f1'] >= -18.66778) & 
       (df_1['f1'] <= 9.050688) &
       (df_1['f0'] >= -27.82962)
])
df_1.shape

(99362, 5)

In [7]:
# Оценим распределение данных в признаках по скважинам в регионе №1
df_2 = pd.read_csv('C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/7. Модель по выбору региона где добывать нефть выгоднее/geo_data_2.csv')
display(df_2.head())
#df_2.info()
#info(df_2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
# Очистим исходные данные от выбрасов по признакам f0, f1, f2
df_2 = (df_2[
       (df_2['f0'] >= -4.64083) & 
       (df_2['f0'] <= 4.637965) &
       (df_2['f1'] >= -4.682104) & 
       (df_2['f1'] <= 4.67061) &
       (df_2['f2'] >= -6.944602) & 
       (df_2['f2'] <= 11.94592)
])
df_2.shape

(98001, 5)

### Вывод
1. Мы изучили исходные данные, удалили выявленные выбросы, которые помешают нам адекватно обучить модель.
2. В процессе анализа распределения признаков региона №1 были обнаружены аномалии. Все скважены региона делятся на 12 классов с по уровню запасов, что является невозможным в природе. 
3. Считаю что данные по региону №1 требую уточнения у геологов - возможно в данных ошибка.

# 2. Обучение и проверка модели<a id='2-bullet'></a>

#### Обучите и проверьте модель для каждого региона:
* 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
* 2.2. Обучите модель и сделайте предсказания на валидационной выборке.
* 2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
* 2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
* 2.5. Проанализируйте результаты.

In [9]:
# Создадим функцию по по разбивке данных на обучающую и валидационную выборки
def neft_model(df):
        
    # Построим матрицу корреляций
    corr = df.corr()
    corr.style.background_gradient(cmap='coolwarm')
    display(corr.style.background_gradient(cmap='coolwarm'))
        
    features = df.drop(['id','product'], axis = 1) # извлечем признаки 
    target = df['product'] # извлечем целевой признак

    # Разделим исходные данные для кроссвалидации на тренировочную и тестовую выборки 75%/25%
    X_train, X_valid, Y_train, Y_valid = train_test_split(
        features, target, test_size=0.25, random_state=4568)
    print(X_train.shape, X_valid.shape, Y_train.shape, Y_valid.shape)
    return X_train, X_valid, Y_train, Y_valid

In [10]:
# Разобъем данные региона №0 и посмотрим корреляцию признаков
X_train_0, X_valid_0, Y_train_0, Y_valid_0 = neft_model(df_0)
#X_train_0.head()

,f0,f1,f2,product
f0,1.000000,-0.440711,-0.003352,0.143597
f1,-0.440711,1.000000,0.001153,-0.192743
f2,-0.003352,0.001153,1.000000,0.480132
product,0.143597,-0.192743,0.480132,1.000000


(74619, 3) (24873, 3) (74619,) (24873,)


In [11]:
# Разобъем данные региона №1 и посмотрим корреляцию признаков
X_train_1, X_valid_1, Y_train_1, Y_valid_1 = neft_model(df_1)
#X_valid_1.head()

,f0,f1,f2,product
f0,1.000000,0.178986,-0.001616,-0.030336
f1,0.178986,1.000000,-0.002808,-0.010228
f2,-0.001616,-0.002808,1.000000,0.999397
product,-0.030336,-0.010228,0.999397,1.000000


(74521, 3) (24841, 3) (74521,) (24841,)


In [12]:
# Разобъем данные региона №2 и посмотрим корреляцию признаков
X_train_2, X_valid_2, Y_train_2, Y_valid_2 = neft_model(df_2)
#Y_train_1.head()

,f0,f1,f2,product
f0,1.000000,0.000652,0.000509,-0.002947
f1,0.000652,1.000000,0.000699,-0.000893
f2,0.000509,0.000699,1.000000,0.441198
product,-0.002947,-0.000893,0.441198,1.000000


(73500, 3) (24501, 3) (73500,) (24501,)


In [13]:
from sklearn.metrics import mean_squared_error
def LRegression(X_train, X_valid, Y_train, Y_valid):
    model = LinearRegression(normalize=True)
    model.fit(X_train, Y_train)
    predictions_valid = model.predict(X_valid)
    result = mean_squared_error(Y_valid, predictions_valid)**0.5
    df = (pd.DataFrame(predictions_valid, Y_valid)
            .reset_index()
            .set_axis(['product', 'predictions'], axis=1, inplace=False))  
    print("Cредний запас предсказанного сырья:", round(df['predictions'].mean(), 4))
    print("Cредний запас реального сырья:", round(df['product'].mean(), 4))
    print("RMSE модели линейной регрессии на валидационной выборке:", round(result, 2))
    return df

In [14]:
# Предсказания модели и реальные данные региона №0
predictions_0 = LRegression(X_train_0, X_valid_0, Y_train_0, Y_valid_0)
predictions_0.head()

Cредний запас предсказанного сырья: 92.7177
Cредний запас реального сырья: 92.403
RMSE модели линейной регрессии на валидационной выборке: 37.78


,product,predictions
0,63.746378,56.207420
1,48.814163,97.897121
2,55.246818,102.022640
3,76.535707,80.388935
4,32.426127,62.403230


In [15]:
# Предсказания модели и реальные данные региона №1
predictions_1 = LRegression(X_train_1, X_valid_1, Y_train_1, Y_valid_1)
predictions_1.head()

Cредний запас предсказанного сырья: 69.0406
Cредний запас реального сырья: 69.0388
RMSE модели линейной регрессии на валидационной выборке: 0.9


,product,predictions
0,30.132364,29.326654
1,137.945408,137.077430
2,53.906522,55.521062
3,26.953261,27.375079
4,3.179103,1.781508


In [16]:
# Предсказания модели и реальные данные региона №2
predictions_2 = LRegression(X_train_2, X_valid_2, Y_train_2, Y_valid_2)
predictions_2.head()

Cредний запас предсказанного сырья: 94.8515
Cредний запас реального сырья: 94.9302
RMSE модели линейной регрессии на валидационной выборке: 40.03


,product,predictions
0,102.196577,63.392930
1,116.734117,95.511872
2,77.954912,63.560661
3,18.643399,102.083554
4,93.359340,100.885557


### Вывод
1. Признак f2 и целевой признак product запасы нефти в скважине коррелированы со значениями от 0.44 до 0.99 по всем регионам что сказывается на обучение модели. 
2. Там где корреляция максимальна - модель не ошибается, это логично для Линейной регресси. Но даже там где среднеквадратичная ошибка равна почти 50% от среднего, среднее предсказаний по региону имеет точность до десятых долей в сравнении со средним реальных данных что отлично.
3. RMSE моделей там где корреляция выражена слабо откровенно плохая, по всей видимости в регионах №0 и №1 показатели запасов сырья имеют большой разброс с чем линейная регрессия несправляется.

# 3. Подготовка к расчёту прибыли<a id='3-bullet'></a>

In [17]:
# Константы
BUDGET = 10000000000
BARREL_CASH = 450000
PROBABILITY_LOSS = 2.5
OIL_WELL = 200

# Рассчитаем достаточный объём сырья для безубыточной разработки одной скважины
V_VOLUME = round(BUDGET / OIL_WELL / BARREL_CASH, 6)
print('Безубыточный объем сырья для разработки скважины:',V_VOLUME,'тыс. баррелей')
print()

# Сравним полученный объём сырья со средним запасом в каждом регионе
print('В регионе № 0 средний запас предсказанного cырья на скважину составляет:', 
      round(predictions_0['predictions'].mean() / V_VOLUME * 100, 2),'% от уровня безубыточности')
print()
print('В регионе № 1 средний запас предсказанного cырья на скважину составляет:', 
      round(predictions_1['predictions'].mean() / V_VOLUME * 100, 2),'% от уровня безубыточности')
print()
print('В регионе № 2 средний запас предсказанного cырья на скважину составляет:', 
      round(predictions_2['predictions'].mean() / V_VOLUME * 100, 2),'% от уровня безубыточности')

Безубыточный объем сырья для разработки скважины: 111.111111 тыс. баррелей

В регионе № 0 средний запас предсказанного cырья на скважину составляет: 83.45 % от уровня безубыточности

В регионе № 1 средний запас предсказанного cырья на скважину составляет: 62.14 % от уровня безубыточности

В регионе № 2 средний запас предсказанного cырья на скважину составляет: 85.37 % от уровня безубыточности


### Вывод
1. Скважина которая приносит 0 дохода/расхода должна распологать запасами 111.111111 тыс. баррелей. 
2. Регион с самыми предсказуемыми нашей моделью запасами обладает самым низким средним запасом нефти на скважиту почти 50% от точки безубыточности. Это резко снизит количество скважин в этом регионе перспективных к разработке.
3. Наша модель исходя из метрики RMSE часто ошибается, но в среднем предсказывает неплохие перспективы в регионах №0 и №2, проанализируем это далее.

# 4. Расчёт прибыли и рисков <a id='4-bullet'></a>

In [18]:
# Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели
def profit(predictions, product):
    # Выберем 200 лучших скважен из выбранных моделью
    sort_well = predictions.sort_values(ascending=False)[:200]
    # Объем реального сырья по 200 лучшим признакам 
    product_well = product['product'][sort_well.index]
    # Прибыль по целевому признаку
    return round(((product_well  - V_VOLUME) * BARREL_CASH).sum(), 2)

In [19]:
# Применим технику Bootstrap с 1000 выборок
state = np.random.RandomState(12345)
def bootstrap(df):    
    values = []
    for i in range(1000):
        search = df['product'].sample(n=500, replace=True, random_state=state)
        predict_model = df['predictions'][search.index]  
        values.append(profit(predict_model, df))
        

    values = pd.Series(values)
    print('Средняя прибыль:',round(values.mean(), 2), 'рублей')
    print('95%-ый доверительный интервал:', round(values.quantile(0.025),2),'-',round(values.quantile(0.975),2),'рублей')
    risk_loss = sum(values.apply(lambda x: 1 if x <= 0 else 0)) / sum(values.apply(lambda x: 1 if x>=0 else 0)) * 100
    print('Вероятность убытков:',round(risk_loss),"%")

In [20]:
# Найдем основные экономические параметры региона №0
bootstrap(predictions_0)

Средняя прибыль: 388753028.27 рублей
95%-ый доверительный интервал: -137364016.15 - 923041449.88 рублей
Вероятность убытков: 8 %


In [21]:
# Найдем основные экономические параметры региона №1
bootstrap(predictions_1)

Средняя прибыль: 460711471.67 рублей
95%-ый доверительный интервал: 80007005.5 - 871170223.21 рублей
Вероятность убытков: 1 %


In [22]:
# Найдем основные экономические параметры региона №2
bootstrap(predictions_2)

Средняя прибыль: 349201562.64 рублей
95%-ый доверительный интервал: -175439638.84 - 872763361.61 рублей
Вероятность убытков: 11 %


# Вывод<a id='5-bullet'></a>
1. Модели по каждому из 3-х регионов позволили нам выбрать скважины с положительной средней прибылью, у 2-х регионов 95% доверительный интервал значительно смещен в убыточную зону, нужно провести работу с заказчиком по поиску признаков лучшего качества для построения более адекватной модели.
2. Вынести вердикт - какой из регионов рекомендовать в разработку невозможно до уточнения признаков региона 1, если не брать в расчет аномалии данных у него лучшие показатели. Нижняя граница доверительно интервала уже находиться в прибыли и средняя прибыль самая большая из 3-х регионов. Вероятность убытков составляет всего 1%.